In [69]:
import pandas as pd
import re
import ast
import gunshot_utils as utils
import importlib
import os
import random
import pandas as pd
import torchaudio

importlib.reload(utils)

<module 'gunshot_utils' from '/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py'>

In [70]:
audio_csv = pd.read_csv('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/Gunshots/csv/edge-collected-gunshot-audio/updated_gunshot_metadata.csv')

In [71]:
audio_csv = audio_csv[['filename', 'firearm', 'caliber', 'gunshot_location_in_seconds', 'num_gunshots']]

In [72]:
LOUDNESS_THRESHOLD = -15

# List to store the valid rows based on loudness
valid_rows = []

for index, row in audio_csv.iterrows():
    try:
        audio_path = row['filename']
        avg_loudness = utils.calculate_loudness(audio_path)
        if avg_loudness > LOUDNESS_THRESHOLD:
            valid_rows.append(row)
    except FileNotFoundError:
        print(f"File not found: {audio_path}")
        continue
    
filtered_df = pd.DataFrame(valid_rows)

/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py:647: UserWarning: PySoundFile failed. Trying audioread instead.
  avg_loudness = rms_db.mean()
/Users/borosabel/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


File not found: 0a07b229-7d2b-4d2b-8f32-c94cbc7b1487_chan0_v0
File not found: 0a07b229-7d2b-4d2b-8f32-c94cbc7b1487_chan0_v1
File not found: 0a07b229-7d2b-4d2b-8f32-c94cbc7b1487_chan1_v0
File not found: 0a07b229-7d2b-4d2b-8f32-c94cbc7b1487_chan1_v1


In [73]:
filtered_df['gunshot_location_in_seconds'] = filtered_df['gunshot_location_in_seconds'].apply(
    lambda x: utils.preprocess_gunshot_times(x)
)

In [74]:
filtered_df

,filename,firearm,caliber,gunshot_location_in_seconds,num_gunshots
0,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.72269841],1
13,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,"[1.60142857, 1.75, 2.07111111, 2.40210884, 2.6...",11
18,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,"[1.75, 2.19632653, 2.65365079, 3.27027211, 3.7...",14
21,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Remington 870,12 guage,[1.62145125],1
41,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Glock 17,9mm,[1.75],1
...,...,...,...,...,...
2143,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.7406576],1
2144,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,[1.66732426],1
2145,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,"[1.53469388, 1.74122449, 2.14659864, 2.3253288...",5
2146,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Ruger 556,.223 mm,"[1.75, 1.94283447, 2.34698413, 2.52979592, 2.7...",5


In [75]:
glock_df = utils.filter_by_firearm(filtered_df, ['glock'])

In [76]:
glock_df.to_pickle('./pkl_data/gunshot_data_glock_only.pkl')

In [ ]:
# Let's also resample the whole music dataset and convert to .wav - only need to do it once.

In [51]:
music_df = pd.read_excel('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Excel/baseline_data_w_topics_w_features.xlsx', engine='openpyxl')

In [65]:
resampled_df = utils.resample_and_convert_to_wav(music_df, 'Path', './Data/music', limit=300)

Processing audio files: 100%|██████████| 300/300 [01:46<00:00,  2.82it/s]
/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.dropna(subset=['Path'])
/Users/borosabel/Documents/Uni/Thesis/PopMusicInformationRetrieval/gunshot_utils.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['file_extension'] = '.wav'


In [66]:
resampled_df.to_pickle('./pkl_data/music_data_resampled_300.pkl')